In [1]:
pip install faker

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

# --- 1. Generate Product/Inventory Data ---
categories = ["Chocolate", "Gummy", "Hard Candy", "Baked Goods"]
suppliers = ["SweetSupplies Inc.", "Cocoa Ventures", "SugarBloom Co."]

products = []
for product_id in range(1, 51):  # 50 products
    category = random.choice(categories)
    products.append({
        "product_id": product_id,
        "product_name": f"{fake.word().capitalize()} {category}",
        "category": category,
        "supplier": random.choice(suppliers),
        "cost_per_unit": round(random.uniform(0.5, 5.0), 2),
        "stock_quantity": random.randint(10, 200),
        "reorder_threshold": random.randint(5, 25)
    })

inventory_df = pd.DataFrame(products)

# --- 2. Generate CRM Data ---
customers = []
for customer_id in range(1, 101):  # 100 customers
    join_date = fake.date_between(start_date="-2y", end_date="today")
    customers.append({
        "customer_id": customer_id,
        "name": fake.name(),
        "email": fake.email(),
        "join_date": join_date,
        "loyalty_tier": random.choice(["Gold", "Silver", "Bronze"]),
        "total_purchases": random.randint(1, 50),
        "favorite_category": random.choice(categories)
    })

crm_df = pd.DataFrame(customers)

# --- 3. Generate POS/Sales Data ---
transactions = []
for transaction_id in range(1, 1001):  # 1000 transactions
    product = random.choice(products)
    customer = random.choice(customers)
    date = fake.date_between(start_date="-1y", end_date="today")
    quantity = random.randint(1, 5)
    revenue = round(quantity * product["cost_per_unit"] * random.uniform(1.5, 3.0), 2)  # Markup
    transactions.append({
        "transaction_id": transaction_id,
        "date": date,
        "product_id": product["product_id"],
        "store_id": random.randint(1, 5),  # 5 stores
        "customer_id": customer["customer_id"],
        "quantity": quantity,
        "revenue": revenue,
        "discount": round(random.uniform(0.0, 0.2), 2)  # 0-20% discount
    })

pos_df = pd.DataFrame(transactions)

# --- Save to CSV ---
inventory_df.to_csv("inventory_data.csv", index=False)
crm_df.to_csv("crm_data.csv", index=False)
pos_df.to_csv("pos_data.csv", index=False)

print("Mock data generated!")

Mock data generated!


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from openpyxl.styles import Font  

# Load the data
pos_data = pd.read_csv('pos_data.csv')
inventory_data = pd.read_csv('inventory_data.csv')

# Merge to get cost information for each sale
merged_data = pos_data.merge(inventory_data, on='product_id', how='left')

# Calculate cost and profit for each transaction
merged_data['total_cost'] = merged_data['quantity'] * merged_data['cost_per_unit']
merged_data['profit'] = merged_data['revenue'] - merged_data['total_cost']

# Define last month's date range
last_month_end = datetime.today().replace(day=1) - timedelta(days=1)
last_month_start = last_month_end.replace(day=1)
last_month_name = last_month_start.strftime("%B %Y")

# Filter for last month's data
merged_data['date'] = pd.to_datetime(merged_data['date'])
last_month_data = merged_data[(merged_data['date'] >= last_month_start) & (merged_data['date'] <= last_month_end)]

# Calculate Key Metrics by Category
report = last_month_data.groupby('category').agg(
    total_revenue=('revenue', 'sum'),
    total_units_sold=('quantity', 'sum'),
    total_profit=('profit', 'sum')
).reset_index()

report['gross_margin_%'] = (report['total_profit'] / report['total_revenue']) * 100

# Format the DataFrame
report['total_revenue'] = report['total_revenue'].round(2)
report['total_profit'] = report['total_profit'].round(2)
report['gross_margin_%'] = report['gross_margin_%'].round(2)

# Create a Pandas Excel writer
with pd.ExcelWriter(f'Monthly_Financial_Report_{last_month_name}.xlsx', engine='openpyxl') as writer:
    report.to_excel(writer, sheet_name='Summary by Category', index=False)
    
    # Access the workbook and the worksheet
    workbook = writer.book
    worksheet = writer.sheets['Summary by Category']
    
    # Add a Title
    title_cell = worksheet.cell(row=1, column=1)
    title_cell.value = f"Confectionately Yours Ventures - Financial Summary ({last_month_name})"
    title_cell.font = Font(bold=True, size=16)

print(f"✅ Report generated: Monthly_Financial_Report_{last_month_name}.xlsx")

✅ Report generated: Monthly_Financial_Report_August 2025.xlsx


In [1]:
import pandas as pd

# Read the transactions
transactions_df = pd.read_csv('inventory_transactions.csv')

# Group by product and sum the quantity to get the current stock
current_stock_df = transactions_df.groupby('product_id')['quantity'].sum().reset_index()
current_stock_df.rename(columns={'quantity': 'current_stock'}, inplace=True)

# Merge with inventory_data to get the product names and thresholds
inventory_df = pd.read_csv('inventory_data.csv')
current_stock_snapshot = current_stock_df.merge(
    inventory_df[['product_id', 'product_name', 'reorder_threshold', 'supplier']], 
    on='product_id', 
    how='left'
)

# Save this new snapshot
current_stock_snapshot.to_csv('current_stock_snapshot.csv', index=False)

print("✅ current_stock_snapshot.csv generated!")

✅ current_stock_snapshot.csv generated!
